In [1]:
import mand.core

from mand.core import Entity, node, Context
from mand.core import PrintMonitor

In [2]:
class O(Entity):

    @node
    def X(self):
        return 10
    
    @node
    def Y(self):
        return self.X()**3
    
    @node
    def eps(self):
        return 1e-4
    
    @node 
    def delta(self):
        eps = self.eps()
        y = self.Y()
        x_bump = self.X() + self.eps()
        c = Context({self.X: x_bump}, name='X-up')
        with c:
            y_bump = self.Y()
        return (y_bump-y)/eps
    
    @node 
    def gamma(self):
        eps = self.eps()
        d = self.delta()
        x_bump = self.X() + self.eps()
        c = Context({self.X: x_bump})
        with c:
            d_bump = self.delta()
        return (d_bump-d)/eps

In [3]:
with PrintMonitor():
    o = O()
    d = o.delta()
    print 'delta:', d

 GetValue begin ctx: Root, key: (<__main__.O object at 0x10d55ff10>, 'O:delta')
   GetValue/Calc begin ctx: Root, key: (<__main__.O object at 0x10d55ff10>, 'O:delta')
adding dep on O@10d55ff10/O:eps
          for O@10d55ff10/O:delta
     GetValue begin ctx: Root, key: (<__main__.O object at 0x10d55ff10>, 'O:eps')
       GetValue/Calc begin ctx: Root, key: (<__main__.O object at 0x10d55ff10>, 'O:eps')
         GetValue/Calc end ctx: Root, key: (<__main__.O object at 0x10d55ff10>, 'O:eps')
       Context set value: 0.0001, ctx: Root, key: (<__main__.O object at 0x10d55ff10>, 'O:eps')
       GetValue end value: 0.0001, ctx: Root, key: (<__main__.O object at 0x10d55ff10>, 'O:eps')
adding dep on O@10d55ff10/O:Y
          for O@10d55ff10/O:delta
     GetValue begin ctx: Root, key: (<__main__.O object at 0x10d55ff10>, 'O:Y')
       GetValue/Calc begin ctx: Root, key: (<__main__.O object at 0x10d55ff10>, 'O:Y')
adding dep on O@10d55ff10/O:X
          for O@10d55ff10/O:Y
         GetValue begin

In [4]:
with PrintMonitor():
    d = o.delta()
    print 'delta:', d
    g = o.gamma()
    print 'gamma:', g

 GetValue begin ctx: Root, key: (<__main__.O object at 0x10d55ff10>, 'O:delta')
   GetValue from ctx value: 300.003000009, ctx: Root, key: (<__main__.O object at 0x10d55ff10>, 'O:delta')
delta: 300.003000009
 GetValue begin ctx: Root, key: (<__main__.O object at 0x10d55ff10>, 'O:gamma')
   GetValue/Calc begin ctx: Root, key: (<__main__.O object at 0x10d55ff10>, 'O:gamma')
adding dep on O@10d55ff10/O:eps
          for O@10d55ff10/O:gamma
     GetValue begin ctx: Root, key: (<__main__.O object at 0x10d55ff10>, 'O:eps')
       GetValue from ctx value: 0.0001, ctx: Root, key: (<__main__.O object at 0x10d55ff10>, 'O:eps')
adding dep on O@10d55ff10/O:delta
          for O@10d55ff10/O:gamma
     GetValue begin ctx: Root, key: (<__main__.O object at 0x10d55ff10>, 'O:delta')
       GetValue from ctx value: 300.003000009, ctx: Root, key: (<__main__.O object at 0x10d55ff10>, 'O:delta')
adding dep on O@10d55ff10/O:X
          for O@10d55ff10/O:gamma
     GetValue begin ctx: Root, key: (<__main__.O

In [5]:
class O2(Entity):

    @node
    def X(self):
        return 10
    
    @node
    def Y(self):
        return self.X()**3
    
    @node
    def eps(self):
        return 1e-4
    
    @node
    def deltaCtx(self):
        x_bump = self.X() + self.eps()
        c = Context({self.X: x_bump}, name='X-up')
        return c
    
    @node 
    def delta(self):
        eps = self.eps()
        y = self.Y()
        x_bump = self.X() + self.eps()
        c = self.deltaCtx()
        with c:
            y_bump = self.Y()
        return (y_bump-y)/eps
    
    @node 
    def gamma(self):
        eps = self.eps()
        d = self.delta()
        x_bump = self.X() + self.eps()
        c = Context({self.X: x_bump})
        with c:
            d_bump = self.delta()
        return (d_bump-d)/eps
    
with PrintMonitor():
    o = O2()
    d = o.delta()
    print 'delta:', d
    g = o.gamma()
    print 'gamma:', g

 GetValue begin ctx: Root, key: (<__main__.O2 object at 0x10d57a810>, 'O2:delta')
   GetValue/Calc begin ctx: Root, key: (<__main__.O2 object at 0x10d57a810>, 'O2:delta')
adding dep on O2@10d57a810/O2:eps
          for O2@10d57a810/O2:delta
     GetValue begin ctx: Root, key: (<__main__.O2 object at 0x10d57a810>, 'O2:eps')
       GetValue/Calc begin ctx: Root, key: (<__main__.O2 object at 0x10d57a810>, 'O2:eps')
         GetValue/Calc end ctx: Root, key: (<__main__.O2 object at 0x10d57a810>, 'O2:eps')
       Context set value: 0.0001, ctx: Root, key: (<__main__.O2 object at 0x10d57a810>, 'O2:eps')
       GetValue end value: 0.0001, ctx: Root, key: (<__main__.O2 object at 0x10d57a810>, 'O2:eps')
adding dep on O2@10d57a810/O2:Y
          for O2@10d57a810/O2:delta
     GetValue begin ctx: Root, key: (<__main__.O2 object at 0x10d57a810>, 'O2:Y')
       GetValue/Calc begin ctx: Root, key: (<__main__.O2 object at 0x10d57a810>, 'O2:Y')
adding dep on O2@10d57a810/O2:X
          for O2@10d57a81